# Chaînes de Markov

## Exercice

Pour cet exercice, nous avons analysé le texte intégral de **Candide, ou l'Optimisme** de *Voltaire* afin de créer un dictionnaire qui se comporte comme une chaîne de Markov, où chaque mot peut être suivi d'un autre mot avec une probabilité `p`.
Pour savoir quels mots suivent quel autre mot avec une certaine probabilité, vous pouvez utiliser `data[mot]` et voir quels mots le suivent.

Par exemple `print(data["leibnitz"])` affiche `{'navait': 0.5, 'ne': 0.5}`

*En partant du mot `leibnitz`, dessinez une chaîne de Markov qui affiche les probabilités d'avoir un mot à la suite d'un autre.*

In [1]:
from functools import reduce

class Candide:
    _data = None
    
    @staticmethod
    def data():
        if Candide._data is None:
            Candide.load_text()
        return Candide._data
    
    @staticmethod
    def clean_word(word):
        return ''.join(l for l in word.lower().strip() if l.isalnum())
    
    @staticmethod
    def load_text():
        with open("candide.txt", "r") as file:
            candide = file.read().split()
            data = {}
            for i in range(len(candide) - 1):
                word = Candide.clean_word(candide[i])
                next_word = Candide.clean_word(candide[i+1])
                if word in data:
                    if next_word in data[word]:
                        data[word][next_word] += 1
                    else:
                        data[word][next_word] = 1
                else:
                    data[word] = {next_word: 1}
            Candide._data = Candide.compute_stats(data)
            
    @staticmethod
    def compute_stats(_data):
        for i in _data.keys():
            _sum = reduce(lambda accumulator, j: accumulator + _data[i][j], _data[i].keys(), 0)
            for j in _data[i]:
                _data[i][j] /= _sum
        return _data

In [2]:
data = Candide.data()

def most_likely(word_data):
    _max = 0
    best = None
    for next_word in word_data.keys():
        if word_data[next_word] > _max:
            best = next_word
            _max = word_data[next_word]
    return best

def create_sentence(first_word, data, depth):
    current_word = first_word
    sentence = first_word
    for i in range(depth):
        current_word = most_likely(data[current_word]) # On appelle most_likely() ici
        sentence = sentence + " " + current_word
    return sentence

create_sentence("leibnitz", data, 105)

'leibnitz navait pas de la vieille je ne pouvait être autrement car il y a été pendu par le plus de la vieille je ne pouvait être autrement car il y a été pendu par le plus de la vieille je ne pouvait être autrement car il y a été pendu par le plus de la vieille je ne pouvait être autrement car il y a été pendu par le plus de la vieille je ne pouvait être autrement car il y a été pendu par le plus de la vieille je ne pouvait être autrement car il y a été pendu par le plus de'

## Exercice

Comme vous le voyez, votre phrase contient beaucoup de répétitions, car certains mots sont très fréquents (comme le mot `et`) et ils créent donc une boucle infinie.

Pour empêcher ceci, faites en sorte que les mots soient choisis aléatoirement tout en respectant la probabilité qu'ils soit choisis. Vous pouvez utiliser une `table de probabilités`

In [3]:
from random import random

data = Candide.data()

def proba_table(word_data, sentence):
    cumulative = 0
    r = random()
    for word in word_data.keys():
        proba = word_data[word]
        if proba + cumulative > r:
            return word
        cumulative += proba

def create_sentence(first_word, data, depth):
    current_word = first_word
    sentence = first_word
    for i in range(depth):
        current_word = proba_table(data[current_word], sentence) # On appelle probal_table() ici
        sentence = sentence + " " + current_word
    return sentence

create_sentence("leibnitz", data, 100)

'leibnitz ne soient bien que le tome xiii du siècle de leurs bouches se remirent messieurs les paraguains mangèrent du château de badajos dieu confonde la moindre valait mieux 2 les persans les convives étaient dassez larges pièces de lui accorda sa victoire mais comment en cet honnête il y reçoit assez des champs entre ses ennemis que la rade le chapitre xxviii page 126 b un vilain cimetière elle court surlechamp mais ils sembrassèrent en europe par lautre à lui dit candide de leffort quil ma donné les trois nègres quatre soldats du repos travaillons sans donner ordre aux procédures'

## Exercice

Votre bon ami `Vladimir` vous propose d'investir `$10000` dans son business de "plantes médicinales".

`Vladimir` est un revendeur très réputé dans votre quartier, cependant vous savez qu'il peut arriver que ses plantes ne soient pas toujours légales et du coup que vous risquez de vous faire arrêter en vous associant à lui.

Vous estimez néanmoins qu'il peut être intéressant d'investir dans ce projet, cependant vous ne voulez pas prendre trop de risques. C'est pourquoi, vous investissez la somme `i`, tel que:

$$ i < 10000 $$

Soit `π`, un nombre qui correspond au pourcentage de votre investissement par rapport à `10000`, tel que:

$$ \pi = \frac{\mathrm{i} }{\mathrm{10000} } $$

Vous savez que tous les jours, votre investissement vous rapportera 
$$  r(\pi) = \pi^2  $$ 


Néanmoins, plus votre investissement est élevé, plus vos collègues vont se méfier de vos rendements, c'est pourquoi, tous les jours vous avez la probabilité `σ(π)` de vous faire arrêter:

$$  \sigma(\pi) = \frac{\mathrm{1} }{\mathrm{1} + e^{\frac{\mathrm{-\pi}}{\mathrm{5}}} } - \frac{\mathrm{1} }{\mathrm{2} }  $$ 

## Créez un chaîne de Markov Monté Carlo qui détermine combien d'argent vous aurez après `T` périodes et quelle est la probabilité que vous vous fassiez arrêter durant ces `T` périodes.

In [4]:
from math import exp
from random import random

def probability_prison(pi):
    return 1/(1+exp(-pi/5)) - 1/2

def run_simulation(pi, proba_prison):
    bank = 0
    for i in range(T):
        r = random()
        if r < proba_prison:
            return -1
        bank += r**2
    return bank

def markov_chain(i, pi, T):
    _simulations = 1000
    proba_prison = probability_prison(pi)
    prisons = 0
    bank = 0
    for i in range(_simulations): # run 100 simulations
        outcome = run_simulation(pi, proba_prison)
        if outcome == -1:
            prisons += 1
        else:
            bank = outcome
    return (prisons/_simulations, bank)
    
i = 1000
T = 30
pi = i / 10000

markov_chain(i, pi, T)

(0.128, 7.818545104644229)